# Notebook: Filter Sentences from Reviews to create Dataset for Annotation

## Packages

In [1]:
from bs4 import BeautifulSoup
from langdetect import detect
import pandas as pd
import json
import nltk
import uuid
from nltk.tokenize import sent_tokenize
import re

## Settings

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nils_hellwig/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Constants

In [3]:
REVIEWS_PATH = "reviews_dataset/reviews.csv"
RANDOM_STATE = 43

## Code

### Load Dataset

In [4]:
df_reviews = pd.read_csv(REVIEWS_PATH)

In [5]:
df_reviews

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization
0,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland","Die LOC schmeckte leicht säuerlich, der kleine...",2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...
1,1,907168709,2005104,0,Wir sind hier nicht in Bayern!,2023-08-01,173stefanh,"München, Deutschland","Wir sind hier nicht in LOC, sind die freundlic...",1.0,berlin,Augustiner am Gendarmenmarkt,de,"Wir sind hier nicht in Bayern, sind die freund..."
2,2,902540630,2005104,0,Heute leider zum abgewöhnen,2023-07-12,Patrick L,"Rüdersdorf, Deutschland",Ich freute mich sehr auf das Wienerschnitzel u...,3.0,berlin,Augustiner am Gendarmenmarkt,de,Ich freute mich sehr auf das Wienerschnitzel u...
3,3,895783388,2005104,0,Teils-Teils,2023-06-18,J9324NNpeterr,"Stellenbosch, Südafrika",Meine Erfahrungen in diesem Jahr waren gemisch...,4.0,berlin,Augustiner am Gendarmenmarkt,de,Meine Erfahrungen in diesem Jahr waren gemisch...
4,4,890917149,2005104,0,schlecht verwaltetes Chaos,2023-05-18,Harald F,"Berlin, Deutschland",ich hätte gerne etwas zum Essen gesagt. Aber e...,1.0,berlin,Augustiner am Gendarmenmarkt,de,ich hätte gerne etwas zum Essen gesagt. Aber e...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3596,3596,889964622,5207419,0,Nicht besonderes,2023-05-11,Mobile28959210843,"Mönchengladbach, Deutschland",Das Essen ist okay mehr aber auch nicht. Ambie...,3.0,mönchengladbach,Brauerei zum Stefanus,de,Das Essen ist okay mehr aber auch nicht. Ambie...
3597,3597,870345760,5207419,0,"Ungebührender Empfang, sehr gutes Essen",2022-12-01,Korich,"Görlitz, Deutschland",Zunächst einmal wurden wir von einem Mann mit ...,4.0,mönchengladbach,Brauerei zum Stefanus,de,Zunächst einmal wurden wir von einem Mann mit ...
3598,3598,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Wir sind nicht zum ersten Mal aber auch defini...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...
3599,3599,863841135,5207419,0,Gutes Essen mit Biergartenambiente,2022-10-09,788heinzn,"Köln, Deutschland","Gutes Essen, Portionen sehr groß (für uns zu g...",4.0,mönchengladbach,Brauerei zum Stefanus,de,"Gutes Essen, Portionen sehr groß (für uns zu g..."


### Filter Sentences

In [6]:
df_reviews_sentences = pd.DataFrame(columns=list(df_reviews.columns) + ['sentence_idx', "id"])

for idx, row in df_reviews.iterrows():
    sentences = sent_tokenize(row['text'], language="german")
    sentence_index = 0
    for sentence in sentences:
        if len(sentence) > 1:
            new_row = row.copy() 
            new_row['text'] = sentence 
            new_row['id'] = str(uuid.uuid4())
            new_row['sentence_idx'] = sentence_index
            df_reviews_sentences.loc[len(df_reviews_sentences)] = new_row
            sentence_index += 1

In [7]:
df_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id
0,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland","Die LOC schmeckte leicht säuerlich, der kleine...",2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...,0,353e2640-984c-47a4-a6b9-e8c97e25c54d
1,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland",Der Schweinebraten war lauwarm und der Kaisers...,2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...,1,3dd1e407-7de8-4099-90ef-4b9331f21a22
2,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland",Ich bin so enttäuschst.,2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...,2,469e0841-7e76-4803-8758-06f8d3fd81b7
3,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland",Gehe ich nicht mehr hin.,2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...,3,2478ca57-d066-414c-a709-f77afd9034bf
4,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland","Schlecht kann ich auch woanders, besonders bei...",2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...,4,2b1c6fae-d30d-453b-8a89-d0191672f365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15344,3600,861397747,5207419,0,Das Schnitzelparadies!,2022-09-22,vereny2020,NaN,Das Essen schmeckt aber generell grandios.,5.0,mönchengladbach,Brauerei zum Stefanus,de,Ich bin im Schnitzelparadies <3 Die Schnitzel ...,1,7a70d2d1-483b-4e01-b5ba-4402e885192b
15345,3600,861397747,5207419,0,Das Schnitzelparadies!,2022-09-22,vereny2020,NaN,Der Beilagensalat ist auch der Kracher: frisch...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Ich bin im Schnitzelparadies <3 Die Schnitzel ...,2,200b8dec-58a6-44e8-9ad8-45f14bd425c6
15346,3600,861397747,5207419,0,Das Schnitzelparadies!,2022-09-22,vereny2020,NaN,"Im Stefanus gibt es auch selbstgebrautes, sehr...",5.0,mönchengladbach,Brauerei zum Stefanus,de,Ich bin im Schnitzelparadies <3 Die Schnitzel ...,3,05e74f3c-4d9a-40f6-948c-459bf8ee22e2
15347,3600,861397747,5207419,0,Das Schnitzelparadies!,2022-09-22,vereny2020,NaN,"Der Service ist sehr nett, zuvorkommend und he...",5.0,mönchengladbach,Brauerei zum Stefanus,de,Ich bin im Schnitzelparadies <3 Die Schnitzel ...,4,a5fe1770-ad11-49d0-90c4-0c97410b119d


### Filter Non-German Sentences

Even if the language code = "de", we have observed that some sentences are in other languagesthan german. These are excluded as well with the help of Google's language-detection.

In [8]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'Unknown'

df_reviews_sentences['detected_language'] = df_reviews_sentences['text'].apply(detect_language)
df_reviews_sentences[df_reviews_sentences['detected_language'] != 'de']

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
17,4,890917149,2005104,0,schlecht verwaltetes Chaos,2023-05-18,Harald F,"Berlin, Deutschland",Es war viel los.,1.0,berlin,Augustiner am Gendarmenmarkt,de,ich hätte gerne etwas zum Essen gesagt. Aber e...,3,1078e6b4-a229-4e1d-bc93-7d0878a4b6dd,es
66,13,866452621,2005104,1,Mind this touri-gap,2022-10-29,452stephand,"Nürnberg, Deutschland",Fazit: mind this touri-gap,1.0,berlin,Augustiner am Gendarmenmarkt,de,Wunderschöne Location mit mangelhafter Leitung...,1,867f6186-e23f-446b-800c-2aa1d472276a,en
93,17,856900319,2005104,1,Deftig bayrisch,2022-08-28,tatrab,"Kaiserslautern, Deutschland",Kann man mal hin,3.0,berlin,Augustiner am Gendarmenmarkt,de,Vorzubereitete Speisen in bayrischer Manier zu...,2,ab9df676-82a8-4e02-abc7-f516a4a91dec,tl
113,22,847064431,2005104,2,Enttäuschend,2022-07-09,HerSchnu,"Erfurt, Deutschland",Fing schon gut an….,2.0,berlin,Augustiner am Gendarmenmarkt,de,In jedem bayerischem Gasthof isst man besser… ...,2,a8a7a7da-59db-4ad1-90a0-88fbdf7a7a68,en
131,25,883653345,947082,0,Tolle Burger!,2023-03-26,dittelbrunn,"Lübeck, Deutschland",Ketchup.,5.0,berlin,Burgermeister,de,Wir waren in der Filiale am S-Bahnhof Eberswal...,2,6d487248-63cf-497e-a736-31687c7f2a76,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15202,3564,866578862,13144674,6,Leckeres Essen in Mönchengladbach,2022-10-30,marcelbF2818DO,NaN,BRAVO!,5.0,mönchengladbach,Frenzen Restaurant & Event,de,Ein tolles Restaurant! Ambiente und Empfang su...,3,900db9c8-d0e7-46ec-b5e4-2d3fb9d4f4c8,pt
15231,3570,862649725,13144674,6,Einfach eine Freude und ein Genuss,2022-09-30,Bernd W,NaN,Klasse.,5.0,mönchengladbach,Frenzen Restaurant & Event,de,"Okay, ich koche gern selbst. Wenn ich aber mal...",4,e213fef3-ba89-458c-927e-9754bf036373,et
15241,3570,862649725,13144674,6,Einfach eine Freude und ein Genuss,2022-09-30,Bernd W,NaN,Place to be.,5.0,mönchengladbach,Frenzen Restaurant & Event,de,"Okay, ich koche gern selbst. Wenn ich aber mal...",14,58e2fc20-c59c-486d-9fe0-276a86e9488c,en
15249,3572,861638375,13144674,6,"Handwerklich einwandfrei, Hohe Qualität und ei...",2022-09-24,Tilmann S,NaN,Top.,5.0,mönchengladbach,Frenzen Restaurant & Event,de,Eher zufällig kehrten wir als Ortsunkundige im...,4,f5c186e1-2213-495e-97ad-53f6f529e8c6,nl


In [9]:
df_reviews_sentences = df_reviews_sentences[df_reviews_sentences['detected_language'] == 'de']

### Balancing

In [10]:
df_balanced_reviews_sentences = df_reviews_sentences.groupby(['rating'], group_keys=False).apply(lambda x: x.sample(800, replace=False, random_state=RANDOM_STATE))
df_balanced_reviews_sentences.reset_index(drop=True, inplace=True)

### Randomisierung

In [11]:
df_balanced_reviews_sentences = df_balanced_reviews_sentences.sample(frac=1, random_state=RANDOM_STATE)  # 'frac=1' mischt den gesamten DataFrame
df_balanced_reviews_sentences.reset_index(drop=True, inplace=True)

### Check If Sentences are Unique

In [12]:
df_balanced_reviews_sentences.groupby(["review_id", "sentence_idx"]).size().reset_index(name="count")

,review_id,sentence_idx,count
0,845636774,0,1
1,845636774,1,1
2,845636774,2,1
3,845636774,3,1
4,845658907,0,1
...,...,...,...
3995,916494891,0,1
3996,916494891,1,1
3997,916494891,3,1
3998,916494891,4,1


### Store as .csv 

In [13]:
df_reviews_sentences.to_csv("reviews_dataset/reviews_sentences.csv")

In [14]:
df_balanced_reviews_sentences.to_csv("reviews_dataset/balanced_reviews_sentences.csv")

In [15]:
df_balanced_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
0,2595,846014268,1041084,0,Sehr nachgelassen…,2022-07-03,Tanja S,"Bielefeld, Deutschland",Waren am Samstag nach längerer Zeit mal wieder...,1.0,bielefeld,Brauhaus Joh. Albrecht,de,Waren am Samstag nach längerer Zeit mal wieder...,0,4cb0f1e4-5f91-4be1-b377-f60152094c4b,de
1,3390,869299715,2049344,0,"Leider nicht zu empfehlen, trotz des sehr schö...",2022-11-21,Haluretube,NaN,hat uns die Geschäftsführerin ohne Begrüßung o...,1.0,wiesbaden,Cafe Maldaner,de,"Wir waren als Paar am Wochenende dort, standen...",5,3dca9649-77b9-4d35-a51d-b62b81301555,de
2,2425,891742980,2719907,0,Muss man nicht haben,2023-05-24,Climber35985073568,"Wülfrath, Deutschland","Was mich aber am meisten stört, ist, dass man ...",3.0,wuppertal,NOI,de,"Das Essen ist mittelmäßig, geschmacklich kein ...",2,8748e672-dccb-45d4-b46b-157a1d585e15,de
3,1527,907496533,743219,0,Nie wieder,2023-08-03,travelexperience62,"Berlin, Deutschland","Waren 2x dort, da uns das Ambiente eigentlich ...",1.0,leipzig,Cafe Riquet,de,"Waren 2x dort, da uns das Ambiente eigentlich ...",0,6b0694f5-5267-4720-adce-1ab3cbeff435,de
4,743,889813531,1155072,0,Gute Frankfurter Äppelwoi-Kneipe,2023-05-10,Don_Mario_1973,"Ingelheim, Deutschland",- sehr gute Auswahl an Frankfurter Speisen- Gr...,5.0,frankfurt,Apfelwein Wagner,de,- sehr gute Auswahl an Frankfurter Speisen- Gr...,0,ba929b82-ef43-4c78-83a4-e1ab7661be1b,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,2949,890699318,1340365,1,Unzufrieden,2023-05-17,heinzmesserschmidt,"Münster, Deutschland",Hatte ein sehniges Teil was nicht zu kauen war...,3.0,münster,Kleiner Kiepenkerl,de,Leider war das Lamm wieder durch obwohl Rare b...,1,c4ca4369-c6d0-41e8-9668-b404ee602855,de
3996,1707,854698398,1884768,1,Super leckeres sehr appetitlich zubereitetes E...,2022-08-17,angelW2292N,NaN,Eine ausgezeichnete Adresse!,5.0,bremen,Kleiner Olymp,de,"Das Essen war hervorragend, die Bedienung supe...",2,9feef3d3-df22-4360-bd93-8f44f0e6c07a,de
3997,3404,906685693,1005252,0,Mit der Hoffnung auf positive Veränderung!,2023-07-30,Rainer Joshua C,NaN,Seit Jahren sind wir mir der Familie und Freun...,3.0,wiesbaden,BENNER´s Bistronomie,de,Seit Jahren sind wir mir der Familie und Freun...,0,a3d1a6cd-4fce-4f54-b2c1-1248e4f55855,de
3998,1417,863058044,2084055,27,Super heute,2022-10-03,Navigate65144012388,NaN,Leckeres Bier und ein guter Wein haben den Abe...,5.0,leipzig,Ratskeller,de,Lecker gegessen! Tolle Bedienung! Vielen Dank!...,5,fc7cb1cf-3cb9-442f-a056-628f14e64e94,de


In [19]:
df_balanced_reviews_sentences["rating"].value_counts()

rating
1.0    800
3.0    800
5.0    800
2.0    800
4.0    800
Name: count, dtype: int64

In [36]:
dfs = [df_balanced_reviews_sentences[df_balanced_reviews_sentences["rating"] == i] for i in range(1, 6)]
df_alternating = pd.DataFrame(columns=df_balanced_reviews_sentences.columns)

max_rows = max(len(df) for df in dfs)
for i in range(max_rows):
    for k in range(5):
        if i < len(dfs[k]):
            df_alternating = pd.concat([df_alternating, dfs[k].iloc[[i]]])

df_balanced_reviews_sentences = df_alternating.reset_index(drop=True)

In [37]:
df_balanced_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
0,1216,893752217,2084055,7,Miserable 😖,2023-06-06,Explorer427892,NaN,"Dafür eine schmierige Sauce, die ungenießbar w...",1.0,leipzig,Ratskeller,de,Wer sich bestrafen möchten bestellt ein Steak ...,2,a288c405-cb5c-4fb2-8c99-77d3171ac611,de
1,83,915894333,1483463,0,Nicht zu empfehlen und ungenießbar,2023-09-10,Markus B,"Weilburg, Deutschland","Das Ganze kostete zudem unverschämte 89,- Euro.",2.0,hamburg,Bullerei,de,Noch nie so reingelegt worden. Das bestellte S...,5,ba5a8383-82e5-48fe-809b-31542dfcd318,de
2,567,909087538,966524,0,Aufforderung zu gehen.,2023-08-09,Quest01755833040,"Saarbrücken, Deutschland",Nach dem wir jedoch 2 mal unaufgefordert und b...,3.0,köln,Cafe Reichard,de,"Café war wirklich sehr gut, der Kuchen gut (ob...",2,3cb710f1-b962-4988-896a-eb9301b29ef1,de
3,1723,869007232,2049557,0,War ein schöner Abend,2022-11-19,anatolij2018,"Lüneburg, Deutschland",Freundliche Bedienung.,4.0,bremen,Schüttinger Gasthausbrauerei,de,"Leckeres ,authentisches Essen. Sehr leckeres B...",2,4d239b86-3ab7-40cc-9c9e-67f43688a6c1,de
4,1368,874174792,2084055,22,Besuch aus Hamburg,2023-01-06,Maps64021963089,"Hamburg, Deutschland",Gerne wieder.,5.0,leipzig,Ratskeller,de,"Super leckeres Essen, sehr freundliche und auf...",1,e596d28e-ff58-4387-9e1f-ae7905d6b7af,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,637,899568262,1190751,3,Schlechter Kundenservice,2023-07-01,Jet62410338264,"Scottsdale, Arizona","Der Kellner teilte uns mit, dass dies ein Rest...",1.0,frankfurt,Trattoria i Siciliani,de,"Ich hielt an, um einen Freund auf einen Drink ...",6,5cfa8211-2167-4c9e-8b8c-afa609d71237,de
3996,447,893365512,718846,0,Leider deutlich schlechter geworden.,2023-06-04,ThorsHammer8,Augsburg,Die Bedienung war nicht recht kommunikativ und...,2.0,münchen,Schneider Bräuhaus,de,Seit vielen Jahren mal wieder hier gewesen und...,1,5e40c8e2-82f7-4b32-a720-a88dd7c30bef,de
3997,303,882297875,695667,1,Leider nichts Besonderes für den Preis,2023-03-15,Kuddel1959,"Oldenburg in Holstein, Deutschland",Im Ergebnis haben weder die Speisen noch der W...,3.0,hamburg,Fischereihafen-Restaurant,de,Aus Anlass eines Jubiläums sollte es hanseatis...,4,0186aa3f-ba22-48e4-8ffb-dea8ca3c4cfa,de
3998,2884,883589289,743080,4,"Herzliche Empfang, gut bürgerliche Küche",2023-03-26,cathrin_talayman,"Bremen, Deutschland","Ob mit Reservierung oder ohne, jeder wird irre...",4.0,münster,Großer Kiepenkerl,de,"Belebter Laden, in dem man von Frau Maar an de...",1,c9c8a58f-132a-48a8-bc4e-51cab984a1e4,de


In [38]:
df_balanced_reviews_sentences["rating"].value_counts()

rating
1.0    800
2.0    800
3.0    800
4.0    800
5.0    800
Name: count, dtype: int64